In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import urllib
import sqlalchemy

In [2]:
# Парсинг условий продажи

# url = 'https://www.tomsk.ru09.ru/realty?subaction=detail&id=4414402'
# test_soup = get_soup_by_url(url)

# for i in df.index[0:10]:
#     test_soup = get_soup_by_url(i)
#     if
#     print(test_soup.find('sup').parent.text)

# get_soup_by_url(df.index[0])
# requests.get(df.index[0])

In [4]:
params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                     "SERVER=OLEG;"
                                     "DATABASE=Apartment_Tomsk;"
                                     "Trusted_Connection=yes")
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={0}".format(params))

In [16]:
def get_soup_by_url(url):
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    
    return soup    

In [6]:
# Получаем номер последней страницы
def get_number_last_page():
    
    soup = get_soup_by_url('https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page=1')
    number_last_page = int(soup.find('td', {'class':'pager_pages'}).find_all('a')[4].text)
    
    return number_last_page

In [7]:
def find_district_field(keys):
    
    for i, j in enumerate(keys):
        if 'район' in j:
            break
    return i

In [8]:
def parse_apartment(url):
    soup = get_soup_by_url(url)
    
    keys = [i.find('span').text.replace('\xa0','').lower() for i in soup.find_all('tr', {'class': 'realty_detail_attr'})]
    
    district_idx = find_district_field(keys)
    items = {'район': keys[district_idx]}
    
    keys = [j for i, j in enumerate(keys) if i not in (district_idx - 1, district_idx)]
    values = [i.text.replace('\xa0', ' ') for i in soup.find_all(class_='nowrap')]
    
    items.update(dict(zip(keys, values)))
    
    items['адрес'] = soup.find(class_='table_map_link').text.replace('\xa0', ' ')
    items['цена'] = int(soup.find('div', {'class': 'realty_detail_price inline'}).text.replace('\xa0','').replace('руб.',''))
    items['ид'] = int(soup.find('strong').text)
    items['дата добавления'] = soup.find(class_='realty_detail_date nobr').get('title')
    items['дата истечения'] = soup.find_all(class_='realty_detail_date')[4].get('title')
    items['ссылка'] = url
    
    return items

In [9]:
def get_urls_pages(start_page=1, end_page=None):
    
    url_base = 'https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page='
    
    end_page = end_page or get_number_last_page()
    pages_to_parse = range(start_page, end_page + 1)
    urls_pages = [url_base + str(i) for i in pages_to_parse]
        
    return urls_pages

In [25]:
def get_urls_apartments_by_page(url_page):
    
    url_base = 'https://www.tomsk.ru09.ru'
    
    soup = get_soup_by_url(url_page)
    soup = soup.find_all('a', {'class':'visited_ads'})

    urls_apartments = {url_base + i.get('href') for i in soup}
    
    return urls_apartments

In [17]:
def main(start_page=1, end_page=None):
    
    rename_map = {'район': 'District',
                 'адрес': 'Address',
                 'вид': 'Sales_Type',
                 'год постройки': 'Year_Building',
                 'материал': 'Material',
                 'этаж/этажность': 'Floor_Numbers_Of_Floors',
                 'этажность': 'Floors_In_Building',
                 'тип квартиры': 'Apartment_Type',
                 'цена': 'Price',
                 'общая площадь': 'Square_Total',
                 'жилая': 'Square_Living',
                 'кухня': 'Square_Kitchen',
                 'количество комнат': 'Rooms_Number',
                 'отделка': 'Apartment_Condition',
                 'санузел': 'Bathroom_Type',
                 'балкон/лоджия': 'Balcony_Loggia',
                 'дата добавления': 'Date_Add',
                 'дата истечения': 'Date_Expiration',
                 'ссылка': 'Url_Link',
                 'ид': 'Id'}
    
    df = pd.DataFrame(columns=list(rename_map.keys()))
    
    urls_in_database = pd.read_sql('SELECT DISTINCT Url_Link FROM Apartment_Tomsk.dbo.Apartments', engine)
    urls_in_database = set(urls_in_database['Url_Link'])
    
    len_storage = len(urls_in_database)
    print('Apartments in storage:', len_storage, '\n')
    
    urls_pages = get_urls_pages(start_page, end_page)
    for url_page in tqdm(urls_pages, desc='Pages'):
        urls_apartments = get_urls_apartments_by_page(url_page)
        urls_apartments_to_parse = urls_apartments.difference(urls_in_database)
        
        if len(urls_apartments_to_parse) != 0:
            for url_apartment in tqdm(urls_apartments_to_parse, desc='Apartments', leave=False):
                df = df.append(parse_apartment(url_apartment), ignore_index=True)
            
    if not df.empty:
        df.rename(columns=rename_map, inplace=True)
        df.to_sql(name='Apartments', con=engine, schema='dbo',  if_exists='append', index=False)
    print('New Apartments:', len(df))

In [18]:
main()

Apartments in storage: 10610 



KeyboardInterrupt: 

In [198]:
pd.read_sql_table('Apartments', engine)

,District,Address,Sales_Type,Year_Building,Material,Floor_Numbers_Of_Floors,Floors_In_Building,Apartment_Type,Price,Square_Total,Square_Living,Square_Kitchen,Rooms_Number,Apartment_Condition,Bathroom_Type,Balcony_Loggia,Date_Add,Date_Expiration,Url_Link,Id
0,кировский район,Ленина (село Тимирязевское),вторичное,NaN,дерево,1/2,2,None,1490000.0,60 кв.м,None,None,2,в хорошем состоянии,совмещенный,None,01.05.2020 13:03:25,30.07.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4417386
1,кировский район,Мокрушина 13,вторичное,2002.0,панель,2/10,10,None,3150000.0,57.45 кв.м,None,10 кв.м,2,в хорошем состоянии,раздельный,"балкон, остекление",29.04.2020 13:36:07,28.07.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4346012
2,кировский район,Полины Осипенко 8а,вторичное,1963.0,кирпич,5/5,5,None,3150000.0,43.6 кв.м,32 кв.м,4.7 кв.м,2,в отличном состоянии,совмещенный,"балкон, остекление",01.05.2020 13:04:34,30.07.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4433464
3,советский район,Сибирская 118,вторичное,2008.0,кирпич,1/9,9,None,3280000.0,67.8 кв.м,50 кв.м,None,2,None,None,None,01.05.2020 13:06:14,15.06.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4255446
4,ленинский район,Ференца Мюнниха 22,вторичное,1978.0,панель,3/9,9,None,3550000.0,64.8 кв.м,None,None,3,в хорошем состоянии,раздельный,"лоджия, остекление",01.05.2020 13:07:06,30.07.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4424538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10605,советский район,Маяковского 20,новостройка,NaN,монолит,11/17,17,None,3660000.0,52.98 кв.м,None,None,2,в хорошем состоянии,совмещенный,"лоджия, остекление",22.05.2020 14:23:11,20.08.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4445106
10606,кировский район,Нахимова 15 (Федора Лыткина 2б),вторичное,1976.0,кирпич,8/9,9,None,2200000.0,28.4 кв.м,15 кв.м,9 кв.м,1,None,совмещенный,None,22.05.2020 15:13:35,20.08.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4140262
10607,советский район,Трамвайная 3,вторичное,1966.0,панель,5/5,5,None,2600000.0,55.1 кв.м,38.7 кв.м,5 кв.м,3,в хорошем состоянии,раздельный,"лоджия, остекление",22.05.2020 13:19:41,20.08.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4445026
10608,ленинский район,Большая Подгорная 118/4,вторичное,NaN,дерево,1/2,2,None,950000.0,30 кв.м,None,None,1,в хорошем состоянии,совмещенный,None,22.05.2020 12:09:10,19.06.2020,https://www.tomsk.ru09.ru/realty?subaction=det...,4363022
